# Strands Agent with AgentCore Memory Tutorial using Hooks

## Overview

This tutorial demonstrates how to build an intelligent personal assistant using Strands agents integrated with AgentCore Memory through hooks. The agent maintains conversation context and learns from interactions to provide personalized responses.

## Tutorial Details

**Use Case**: Personal Assistant with Memory

You'll learn to:
- Set up AgentCore Memory with conversation summaries
- Create memory hooks for automatic storage and retrieval
- Build a Strands agent with persistent memory
- Test memory functionality across conversations

## Tutorial Architecture

![architecture](architecture.png)

## Tutorial Key Features

- **Automatic Memory Storage**: Conversations are automatically saved
- **Context Retrieval**: Previous conversations inform current responses
- **Summary Generation**: Key information is extracted and summarized
- **Tool Integration**: Calculator tool for mathematical operations

## Prerequisites

- AWS account with appropriate permissions
- AgentCore Memory role ARN configured
- Python environment with required packages

## Step-by-Step Tutorial

### Step 1: Environment set up
Let's begin importing all the necessary libraries and defining the clients to make this notebook work.

In [2]:
!pip install -qr requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 1.40.37 requires botocore==1.38.38, but you have botocore 1.39.5 which is incompatible.
fastapi 0.115.13 requires starlette<0.47.0,>=0.40.0, but you have starlette 0.47.1 which is incompatible.
safety-schemas 0.0.14 requires pydantic<2.10.0,>=2.6.0, but you have pydantic 2.11.7 which is incompatible.


In [4]:
from BedrockAgentcore.memory import MemoryClient
from BedrockAgentcore.memory.constants import StrategyType

In [5]:
import logging
from strands import Agent
from datetime import datetime
from typing import Dict, List
from strands_tools import calculator
from strands.hooks.registry import HookProvider, HookRegistry
from strands.hooks.events import MessageAddedEvent, AfterInvocationEvent


# Setup logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger("memory-tutorial")

# Configuration - TBChanged
REGION = "us-west-2"
ROLE_ARN = "<<INSERT-YOUR-IAM-ROLE>>"
ACTOR_ID = f"actor-{datetime.now().strftime('%Y%m%d%H%M%S')}"
SESSION_ID = f"tutorial-{datetime.now().strftime('%Y%m%d%H%M%S')}"

### Step 2: Create Memory Resource

In this step, we're creating our memory resource with a summary strategy. This resource will store and organize our conversation data. The strategy we're defining will automatically generate summaries of conversations and store them in organized namespaces.

In [6]:
# Initialize Memory Client
client = MemoryClient(region_name=REGION, environment="prod")
memory_name = "TutorialMemory"
# Define memory strategy for conversation summaries
strategies = [
    {
        StrategyType.SUMMARY.value: {
            "name": "conversation_summary",
            "description": "Captures summaries of conversations",
            "namespaces": ["summaries/{actorId}/{sessionId}"]
        }
    }
]

# Create memory resource
try:
    memory = client.create_memory_and_wait(
        name=memory_name,
        strategies=strategies,
        description="Memory for tutorial agent",
        event_expiry_days=30,
        memory_execution_role_arn=ROLE_ARN,
    )
    memory_id = memory['memoryId']
    print(f"✅ Created memory: {memory_id}")
except Exception as e:
    if "already exists" in str(e):
        memories = client.list_memories()
        memory_id = next((m['id'] for m in memories if m['name'] == memory_name), None)
        print(f"✅ Using existing memory: {memory_id}")
    else:
        raise e

2025-07-14 04:40:27,371 - INFO - Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
2025-07-14 04:40:27,433 - INFO - Initialized MemoryClient for prod in us-west-2
2025-07-14 04:40:28,179 - INFO - Created memory: TutorialMemory-t7bn1bCf1B
2025-07-14 04:40:28,180 - INFO - Created memory TutorialMemory-t7bn1bCf1B, waiting for ACTIVE status...
2025-07-14 04:43:13,896 - INFO - Memory TutorialMemory-t7bn1bCf1B is now ACTIVE (took 165 seconds)


✅ Created memory: TutorialMemory-t7bn1bCf1B


### Step 3: Create Memory Hook Provider

This step defines our custom `MemoryHookProvider` class that automates memory operations. Hooks are special functions that run at specific points in an agent's execution lifecycle. The memory hook we're creating serves two primary functions:

1. **Retrieve Memories**: Automatically fetches relevant past conversations when a user sends a message
2. **Save Memories**: Stores new conversations after the agent responds

This creates a seamless memory experience without manual management.

In [ ]:
class MemoryHookProvider(HookProvider):
    """Hook provider for automatic memory management"""
    
    def __init__(self, memory_id: str, client: MemoryClient, actor_id: str, session_id: str):
        self.memory_id = memory_id
        self.client = client
        self.actor_id = actor_id
        self.session_id = session_id
        self.namespace = f"summaries/{self.actor_id}/{self.session_id}"
    
    def retrieve_memories(self, event: MessageAddedEvent):
        """Retrieve relevant memories before processing user message"""
        messages = event.agent.messages
        if messages[-1]["role"] == "user" and "toolResult" not in messages[-1]["content"][0]:
            user_message = messages[-1]["content"][0]["text"]
            
            try:
                # Retrieve relevant memories
                memories = self.client.retrieve_memories(
                    memory_id=self.memory_id,
                    namespace=self.namespace,
                    query=user_message
                )
                
                # Extract memory content
                memory_context = []
                for memory in memories:
                    if isinstance(memory, dict):
                        content = memory.get('content', {})
                        if isinstance(content, dict):
                            text = content.get('text', '').strip()
                            if text:
                                memory_context.append(text)
                
                # Inject memories into user message
                if memory_context:
                    context_text = "\n".join(memory_context)
                    original_text = messages[-1]["content"][0]["text"]
                    messages[-1]["content"][0]["text"] = (
                        f"{original_text}\n\nPrevious context: {context_text}"
                    )
                    logger.info(f"Retrieved {len(memory_context)} memories")
                    
            except Exception as e:
                logger.error(f"Failed to retrieve memories: {e}")
    
    def save_memories(self, event: AfterInvocationEvent):
        """Save conversation after agent response"""
        try:
            messages = event.agent.messages
            if len(messages) >= 2:
                # Get last user and assistant messages
                user_msg = None
                assistant_msg = None
                
                for msg in reversed(messages):
                    if msg["role"] == "assistant" and not assistant_msg:
                        assistant_msg = msg["content"][0]["text"]
                    elif msg["role"] == "user" and not user_msg and "toolResult" not in msg["content"][0]:
                        user_msg = msg["content"][0]["text"]
                        break
                
                if user_msg and assistant_msg:
                    # Save conversation
                    self.client.save_conversation(
                        memory_id=self.memory_id,
                        actor_id=self.actor_id,
                        session_id=self.session_id,
                        messages=[(user_msg, "USER"), (assistant_msg, "ASSISTANT")]
                    )
                    logger.info("Saved conversation to memory")
                    
        except Exception as e:
            logger.error(f"Failed to save memories: {e}")
    
    def register_hooks(self, registry: HookRegistry) -> None:
        """Register memory hooks"""
        registry.add_callback(MessageAddedEvent, self.retrieve_memories)
        registry.add_callback(AfterInvocationEvent, self.save_memories)
        logger.info("Memory hooks registered")

### Step 4: Create Agent with Memory

Now we're creating our Strands agent and connecting it with our memory hook provider. This agent will have two key capabilities:

1. **Memory Integration**: The memory hooks we created will enable automatic context retrieval
2. **Calculator Tool**: The agent can perform mathematical operations when needed

This combination creates a personal assistant that both remembers past interactions and can perform useful calculations.

In [ ]:
# Create memory hook provider
memory_hooks = MemoryHookProvider(
    memory_id=memory_id,
    client=client,
    actor_id=ACTOR_ID,
    session_id=SESSION_ID
)

# Create agent with memory hooks and calculator tool
agent = Agent(
    hooks=[memory_hooks],
    tools=[calculator],
    system_prompt="You are a helpful personal math assistant."
)

print("✅ Agent created with memory hooks.")

### Step 5: Test Memory Functionality

In this section, we'll test the agent's memory capabilities through a series of interactions. We'll observe how the agent builds context over time and recalls previous interactions.

First, let's introduce ourselves to the agent and ask a math question:

In [ ]:
# First interaction - introduce yourself
response1 = agent("Hi, I'm John and I love mathematics. What's 15 * 8?")
print(f"Agent: {response1}")

Now, let's see if the agent remembers who we are:

In [ ]:
# Second interaction - test memory recall
response2 = agent("What's my name and what do I like?")
print(f"Agent: {response2}")

Let's give the agent another calculation task:

In [ ]:
# Third interaction - another calculation
response3 = agent("Can you help me with 25 + 17?")
print(f"Agent: {response3}")

Finally, let's check if the agent remembers our calculation history:

In [ ]:
# Fourth interaction - test context awareness
response4 = agent("What calculations have we done together?")
print(f"Agent: {response4}")

### Step 6: Verify Memory Storage

As a final step, we'll verify that our conversations have been properly stored in AgentCore Memory. This demonstrates that the memory hooks are working correctly and the agent can access this information in future interactions.

In [ ]:
# Check stored memories
try:
    memories = client.retrieve_memories(
        memory_id=memory_id,
        namespace=f"summaries/{ACTOR_ID}/{SESSION_ID}",
        query="mathematics calculations"
    )
    
    print(f"\n📚 Found {len(memories)} memories:")
    for i, memory in enumerate(memories, 1):
        if isinstance(memory, dict):
            content = memory.get('content', {})
            if isinstance(content, dict):
                text = content.get('text', '')[:200] + "..."
                print(f"{i}. {text}")
                
except Exception as e:
    print(f"Error retrieving memories: {e}")

## Clean Up

### Optional: Delete Memory Resource

After completing the tutorial, you may want to delete the memory resource to avoid incurring unnecessary costs. The following code is provided for cleanup but is commented out by default.

In [ ]:
# Uncomment to delete the memory resource
# try:
#     client.delete_memory(memory_id=memory_id)
#     print(f"✅ Deleted memory resource: {memory_id}")
# except Exception as e:
#     print(f"Error deleting memory: {e}")

print("Tutorial completed! 🎉")
print("\nKey takeaways:")
print("- Memory hooks automatically store and retrieve conversation context")
print("- Agents can maintain state across multiple interactions")
print("- AgentCore Memory provides semantic search for relevant context")
print("- Tools can be combined with memory for enhanced functionality")

This tutorial has demonstrated how to create an intelligent personal assistant with memory capabilities using Strands agents and AgentCore Memory. You've learned how to set up memory resources, create hooks for automatic memory management, and test the agent's ability to maintain context across multiple interactions.